# Evaluate likelihood ratio and posterior

In [1]:
import sys, os
sys.path.append('../')

import logging
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.metrics import roc_curve

from simulation.population_sim import SubhaloSimulator
from inference.estimator import Estimator
from inference.utils import s_from_r, shuffle

logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

## Evaluate model on train and test data

In [2]:
x_train = np.load("../data/samples/x_train.npy")
y_train = np.load("../data/samples/y_train.npy")
theta_train = np.load("../data/samples/theta_train.npy")

x_test = np.load("../data/samples/x_test.npy")
y_test = np.load("../data/samples/y_test.npy")
theta_test = np.load("../data/samples/theta_test.npy")

In [3]:
x_train, y_train, theta_train = shuffle(x_train, y_train, theta_train)
x_train = x_train[:1000]
y_train = y_train[:1000]
theta_train = theta_train[:1000]

x_test, y_test, theta_test = shuffle(x_test, y_test, theta_test)
print(np.sum(y_train) / len(y_train), np.sum(y_test) / len(y_test))

0.5 0.5


## ROC curve

In [23]:
def make_roc_curves(name):
    re = Estimator()
    re.load("../data/models/" + name)
    
    llr_train, _ = re.evaluate_ratio(x=x_train, theta0=theta_train, test_all_combinations=False)
    s_train = s_from_r(np.exp(llr_train))
    
    llr_test, _ = re.evaluate_ratio(x=x_test, theta0=theta_test, test_all_combinations=False)
    s_test = s_from_r(np.exp(llr_test))
    
    fpr_train, tpr_train, _ = roc_curve(y_train, s_train)
    fpr_test, tpr_test, _ = roc_curve(y_test, s_test)
    
    return fpr_train, tpr_train, fpr_test, tpr_test

In [ ]:
fpr_train_alice, tpr_train_alice, fpr_test_alice, tpr_test_alice = make_roc_curves("alice")
fpr_train_carl, tpr_train_carl, fpr_test_carl, tpr_test_carl = make_roc_curves("carl")

In [ ]:
fig = plt.figure(figsize=(5,5))

plt.plot([0.,1.], [1.,0.], ls="--", c="0.5")
plt.plot(1. - fpr_train_carl, tpr_train_carl, c="C0", ls="--", label="CARL train")
plt.plot(1. - fpr_test_carl, tpr_test_carl, c="C0", ls="-", label="CARL test")
plt.plot(1. - fpr_train_alice, tpr_train_alice, c="C1", ls="--", label="ALICE train")
plt.plot(1. - fpr_test_alice, tpr_test_alice, c="C1", ls="-", label="ALICE test")

plt.legend()

plt.xlabel("1 - fpr")
plt.ylabel("tpr")

plt.tight_layout()
plt.show()


## Evaluate on grid

In [4]:
grid_res = 25

alpha_test = np.linspace(200., 800., grid_res)
beta_test = np.linspace(-1.3, -3.7, grid_res)

theta0, theta1 = np.meshgrid(alpha_test, beta_test)
theta_grid = np.vstack((theta0.flatten(), theta1.flatten())).T

In [ ]:
re = Estimator()
re.load("../data/models/carl")

nllr, _ = re.evaluate_ratio(x=x_train[:20], theta0=theta_grid)
nllr = -2. * nllr

In [ ]:
for i in range(20):
    i_ml = np.argmin(nllr[:,i])
    print(theta_grid[i_ml], nllr[i_ml,i])

In [ ]:
fig = plt.figure(figsize=(10, 4 * 10))

for i in [9,0,1,2,3,4,5,6,7,8]:
    
    # Plot image
    if i == 9:
        ax_left = plt.subplot(10, 2, i*2+1)
        ax = ax_left
    else:
        ax = plt.subplot(10, 2, i*2+1, sharex=ax_left)
    
    plt.imshow(
        np.log10(x_train[i]),
        vmax=3.5,
        cmap='gist_gray',
        extent=(-2.,2.,-2.,2.),
        origin="lower"
    )
    
    plt.xlim(-2.,2.)
    plt.ylim(-2.,2.)
    
    if i == 8:
        plt.xlabel(r"$\theta_x$ [arcsec]")
    else:
        plt.setp(plt.gca().get_xticklabels(), visible=False)
    plt.ylabel(r"$\theta_y$ [arcsec]")
    
    
    
    # Plot estimated likelihood ratio for this image
    if i == 9:
        ax_right = plt.subplot(10, 2, i*2+2)
        ax = ax_right
    else:
        ax = plt.subplot(10, 2, i*2+2, sharex=ax_right)
        
    nllr_ = nllr[:, i]

    best_fit = theta_grid[np.argmin(nllr_)]
    cmin, cmax = -.1, 0.1

    bin_size = alpha_test[1] - alpha_test[0]
    alpha_edges = np.linspace(alpha_test[0] - bin_size/2, alpha_test[-1] + bin_size/2, grid_res + 1)
    bin_size = beta_test[1] - beta_test[0]
    beta_edges = np.linspace(beta_test[0] - bin_size/2, beta_test[-1] + bin_size/2, grid_res + 1)

    pcm = ax.pcolormesh(
        alpha_edges,
        beta_edges,
        nllr_.reshape((grid_res, grid_res)),
        norm=matplotlib.colors.Normalize(vmin=cmin, vmax=cmax),
        cmap='PRGn'
    )
    cbar = fig.colorbar(pcm, ax=ax, extend='both')

    plt.scatter(best_fit[0], best_fit[1], s=80., color='black', marker='*')

    if i == 9:
        plt.xlabel(r'$\alpha$')
    else:
        plt.setp(plt.gca().get_xticklabels(), visible=False)
    plt.ylabel(r'$\beta$')
    cbar.set_label(r'$-2\; \log \;\hat{r}(x | \alpha, \beta)$')

plt.tight_layout()
plt.savefig("../figures/carl_predictions.pdf")

## Evaluate expected LLR on grid

In [29]:
grid_res = 25

alpha_test = np.linspace(200., 800., grid_res)
beta_test = np.linspace(-1.3, -3.7, grid_res)

theta0, theta1 = np.meshgrid(alpha_test, beta_test)
theta_grid = np.vstack((theta0.flatten(), theta1.flatten())).T

In [ ]:
re = Estimator()
re.load("../data/models/carl")

nllr, _ = re.evaluate_ratio(x=x_test, theta0=theta_grid)
expected_nllr = -2. * np.sum(nllr, axis=1)

In [ ]:
fig = plt.figure(figsize=(5, 5))

best_fit = theta_grid[np.argmin(expected_nllr)]
cmin, cmax = -10., 10.

bin_size = alpha_test[1] - alpha_test[0]
alpha_edges = np.linspace(alpha_test[0] - bin_size/2, alpha_test[-1] + bin_size/2, grid_res + 1)
bin_size = beta_test[1] - beta_test[0]
beta_edges = np.linspace(beta_test[0] - bin_size/2, beta_test[-1] + bin_size/2, grid_res + 1)

pcm = ax.pcolormesh(
    alpha_edges,
    beta_edges,
    expected_nllr.reshape((grid_res, grid_res)),
    norm=matplotlib.colors.Normalize(vmin=cmin, vmax=cmax),
    cmap='PRGn'
)
cbar = fig.colorbar(pcm, ax=ax, extend='both')

plt.scatter(best_fit[0], best_fit[1], s=80., color='black', marker='*')

plt.xlabel(r'$\alpha$')
plt.ylabel(r'$\beta$')
cbar.set_label(r'$-2\; E[\log \;\hat{r}(x | \alpha, \beta)]$')

plt.tight_layout()
plt.savefig("../figures/alice_expected_llr.pdf")